# Axiverse Vacua

## Load packages

In [24]:
using Combinatorics
using ArbNumerics
using GenericLinearAlgebra
using LinearAlgebra
using TimerOutputs
using Optim
using LineSearches
using Tullio
using Nemo
using Random
using Distributions

## Function definitions

In [59]:
"""
    pseudo_Qprime(h11,tri,cy)
Generates a Q-matrix with similar properties as database Qs
"""
function pseudo_Qprime(h11,tri,cy=1)
    Qh11 = Matrix{Int}(I,h11,h11)
    Q4 = rand(-5:5,(4,h11))
    Q = vcat(Qh11,Q4)
    return Q
end
"""
    pseudo_Lprime(h11,tri,cy)
Generates vector of hierarchical instanton scales

Returns:
 - vector of length h11+4
 - 1st entry is 1
"""
function pseudo_Lprime(h11,tri,cy=1)
    Ltemp::Vector{ArbFloat} = sort(vcat([1,[rand(ArbFloat)*10. ^-(5*(j-1)) for j=2:h11+4]...]),rev=true)
    return Ltemp
end

"""
    pseudo_Lprime_shuffle(h11,tri,cy)
Generates vector of hierarchical instanton scales

Returns:
 - vector of length h11+4
 - 1st entry is 1
 - following entries shuffled randomly
"""
function pseudo_Lprime_shuffle(h11,tri,cy=1)
    Ltemp::Vector{ArbFloat} = sort(vcat([1,[rand(ArbFloat)*10. ^-(5*(j-1)) for j=2:h11+4]...]),rev=true)
    return vcat(Ltemp[1]...,shuffle(Ltemp[2:end])...)
end

"""
    pseudo_Lprime_mixed(h11,tri,cy)
Generates vector of mixed non-hierarchical and hierarchical instanton scales

Returns:
 - vector of length h11+4
 - 1st entry is 1
 - following entries shuffled randomly
"""
function pseudo_Lprime_mixed(h11,tri,cy=1)
    Ltemp::Vector{ArbFloat} = sort(vcat([1,0.9,0.8,[rand(ArbFloat)*10. ^-(5*(j-1)) for j=4:h11+4]...]),rev=true)
    return Ltemp
end

pseudo_Lprime_mixed

In [3]:
"""
    vacua_out(h11,Q,L)
Computes analytic estimate of number of vacua as a ratio of the volumes of the 
exact symmetries of the potential (θparallel) and the approximate symmetries of the leading
potential inv(Qtilde)

Returns:
 - number of vacua
 - volume of potential
 - volume of lattice containing minima in fundamental domain
"""
function vacua_out(h11::Int,QV::Matrix,LV::Vector)
    S, T, U = snf_with_transform(matrix(Nemo.ZZ,QV))
    Tparallel = Int.(inv(T)[:,1:h11])
    θparalleltest = inv(transpose(QV) * QV) * transpose(QV) * Tparallel
    P = h11+4
    while rank(QV[1:P,:])!=h11
        P+=1
    end
    Lp, Qp = LV[1:P], QV[1:P,:]
    Qtilde = Qp[1,:]
    Qdtilde = zeros(size(Qp[1,:]))
    for i=2:size(Qp,1)
        S = MatrixSpace(Nemo.ZZ, size(Qtilde,1), (size(Qtilde,2)+1))
        m = S(hcat(Qtilde,Qp[i,:]))
        (d,bmat) = Nemo.nullspace(m)
        if d == 0
            Qtilde = hcat(Qtilde,Qp[i,:])
        else
            Qdtilde = hcat(Qdtilde,Qp[i,:])
        end
    end
    vacua = round(abs(det(θparalleltest)) / det(inv(Qtilde)))
    return abs(vacua), Rational.(round.(θparalleltest; digits=10)), Qtilde
end

vacua_out

In [4]:
###################################
#### Code for profiling ###########
###################################

"""
    minimiser_timing(h11,tri,cy,LV,QV,x0,algo)
Optimization function for a particular geometry labelled by `h11`, `tri`, and `cy`.  Takes instanton scales, `Λ`, 
and instanton charges, `Q`, as inputs, as well as some initialisation coordinates `x0`.  Requires specification
of optimisation algorithm _with hessian_, `algo`, _i.e._ `Newton()` or `BFGS()`

Also computes analytic estimate of number of vacua.

Returns:
 - Sign of location of minimum found (mod 2π)
 - log10 of location of minimum found (mod 2π)
 - Matrix of approximate locations of minima (analytic)
"""
function minimiser_timing(h11::Int,tri::Int,cy::Int,LV::Vector,QV::Matrix{Int},x0::Vector,algo)
    reset_timer!()
    """
        vacua_out(h11,Q,L)
    Computes analytic estimate of number of vacua as a ratio of the volumes of the 
    exact symmetries of the potential (θparallel) and the approximate symmetries of the leading
    potential inv(Qtilde)
    
    Returns:
     - number of vacua
     - volume of potential
     - volume of lattice containing minima in fundamental domain
    """
    @timeit "vacua_analytic" function vacua_out(h11::Int,QV::Matrix,LV::Vector)
        S, T, U = snf_with_transform(matrix(Nemo.ZZ,QV))
        Tparallel = Int.(inv(T)[:,1:h11])
#         Tparallel = convert(Matrix{Int},Tparallel)
        θparalleltest = inv(transpose(QV) * QV) * transpose(QV) * Tparallel
        P = h11+4
        while rank(QV[1:P,:])!=h11
            P+=1
        end
        Lp, Qp = LV[1:P], QV[1:P,:]
        Qtilde = Qp[1,:]
        Qdtilde = zeros(size(Qp[1,:]))
        for i=2:size(Qp,1)
            S = MatrixSpace(Nemo.ZZ, size(Qtilde,1), (size(Qtilde,2)+1))
            m = S(hcat(Qtilde,Qp[i,:]))
            (d,bmat) = Nemo.nullspace(m)
            if d == 0
                Qtilde = hcat(Qtilde,Qp[i,:])
            else
                Qdtilde = hcat(Qdtilde,Qp[i,:])
            end
        end
        vacua = round(abs(det(θparalleltest)) / det(inv(Qtilde)))
        return abs(vacua), Rational.(round.(θparalleltest; digits=10)), Qtilde
    end
    Qx = zeros(ArbFloat,size(QV,1));
    """
        fitness(x)
    This is the potential that is optimized at with some initial starting point x
    """
    @timeit "V" function fitness(x::Vector)
        if size(x,1)!=h11
            return "Error, check x"
        end
        V = dot(LV,(1 .- cos.(LinearAlgebra.mul!(Qx,QV,x))))
        return V
    end
    """
        g!(x)
    Updatable gradient of the potential at a point x for optimiser
    """
    @timeit "gradient" function g!(storage::Vector, x::Vector)
        storage .= vcat([dot(LV,QV[:,i] .* sin.(LinearAlgebra.mul!(Qx,QV,x))) for i ∈ 1:h11]...)
    end
    """
        hess(x)
    Hessian evaluated at x to verify eigvals(H(x))>0
    """
    @timeit "hess_check" function hess(x::Vector)
        grad2 = zeros(ArbFloat,(h11,h11))
        hind1 = [[x,y]::Vector{Int64} for x=1:h11,y=1:h11 if x>=y]
        grad2_temp = zeros(ArbFloat,size(hind1,1))
        grad2_temp1 = zeros(Float64,size(LV,1),size(hind1,1))
        @tullio Qx[c] = QV[c,i] * x[i]
        @tullio grad2_temp1[c,k] = @inbounds(begin
        i,j = hind1[k]
                QV[c,i] * QV[c,j] * cos.(Qx[c]) end) grad=false fastmath=false
        @tullio grad2_temp[k] = grad2_temp1[c,k] * LV[c]
        @inbounds for i=1:size(hind1,1)
            j,k = hind1[i]
            grad2[j,k] = grad2_temp[i]
        end
        hessfull = Hermitian(grad2 + transpose(grad2) - Diagonal(grad2))
    end
    """
        hess!(x)
    Updatable version of hessian for optimiser
    """
    @timeit "hess_test" function hess!(hessian::Matrix, x::Vector)
        grad2 = zeros(ArbFloat,(h11,h11))
        hind1 = [[x,y]::Vector{Int64} for x=1:h11,y=1:h11 if x>=y]
        grad2_temp = zeros(ArbFloat,size(hind1,1))
        grad2_temp1 = zeros(Float64,size(LV,1),size(hind1,1))
        @timeit "Qx" begin 
            @tullio Qx[c] = QV[c,i] * x[i]
        end
        @timeit "grad2Q" begin
            @tullio grad2_temp1[c,k] = @inbounds(begin
        i,j = hind1[k]
                QV[c,i] * QV[c,j] * cos.(Qx[c]) end) grad=false
        end
        @timeit "grad2LQ" begin @tullio grad2_temp[k] = grad2_temp1[c,k] * LV[c]
        end
        @timeit "grad2" @inbounds for i=1:size(hind1,1)
            j,k = hind1[i]
            grad2[j,k] = grad2_temp[i]
        end
        @timeit "storage" hessian .= grad2 + transpose(grad2) - Diagonal(grad2)
    end
    """
        grad(x)
    Gradient of potential at x for verifying G(x)>0 at minimum
    """
    grad(x) = vcat([dot(LV,QV[:,i] .* sin.(LinearAlgebra.mul!(Qx,QV,x))) for i ∈ 1:h11]...)
    #################
    ### Optimiser ###
    #################
    @timeit "optim" res = optimize(fitness,g!,hess!,
                x0, algo,
                Optim.Options(x_tol =minimum(abs.(LV)),g_tol =minimum(abs.(LV)),successive_f_tol = 5))
    Vmin = Optim.minimum(res)
    xmin = Optim.minimizer(res)
    #####################
    ## Verify minimum ###
    ### and output ######
    #####################
    if minimum(eigen(hess(xmin)).values) > 0 && maximum(abs.(grad(xmin))) < 1e-10
        @timeit "vacua" vacua, θparalleltest, Qtilde = vacua_out(h11, QV, LV)
        @timeit "a" a = mod.(((θparalleltest * xmin)/2π),1)
        a_sign::Vector{Int} = sign.(a)
        a_log::Vector{Float64} = log10.(abs.(a))
        Vmin_sign::Int = sign(Vmin)
        Vmin_log::Float64 = log10(abs(Vmin))
        xmin_log::Vector{Float64} = log10.(abs.(xmin))
        xmin_sign::Vector{Int} = sign.(xmin)
        print_timer()
        a_sign, a_log, θparalleltest
    end
end



minimiser_timing

In [5]:
"""
    minimiser(h11,tri,cy,LV,QV,x0,algo)
Optimization function for a particular geometry labelled by `h11`, `tri`, and `cy`.  Takes instanton scales, `Λ`, 
and instanton charges, `Q`, as inputs, as well as some initialisation coordinates `x0`.  Requires specification
of optimisation algorithm _with hessian_, `algo`, _i.e._ `Newton()` or `BFGS()`

Also computes analytic estimate of number of vacua.

Returns:
 - Sign of location of minimum found (mod 2π)
 - log10 of location of minimum found (mod 2π)
 - Matrix of approximate locations of minima (analytic)
"""
function minimiser(h11::Int,tri::Int,cy::Int,LV::Vector,QV::Matrix{Int},x0::Vector,algo)
    
    Qx = zeros(ArbFloat,size(QV,1));
    """
        fitness(x)
    This is the potential that is optimized at with some initial starting point x
    """
    function fitness(x::Vector)
        if size(x,1)!=h11
            return "Error, check x"
        end
        V = dot(LV,(1 .- cos.(LinearAlgebra.mul!(Qx,QV,x))))
        return V
    end
    """
        g!(x)
    Updatable gradient of the potential at a point x for optimiser
    """
    function g!(storage::Vector, x::Vector)
        storage .= vcat([dot(LV,QV[:,i] .* sin.(LinearAlgebra.mul!(Qx,QV,x))) for i ∈ 1:h11]...)
    end
    """
        hess(x)
    Hessian evaluated at x to verify eigvals(H(x))>0
    """
    function hess(x::Vector)
        grad2 = zeros(ArbFloat,(h11,h11))
        hind1 = [[x,y]::Vector{Int64} for x=1:h11,y=1:h11 if x>=y]
        grad2_temp = zeros(ArbFloat,size(hind1,1))
        grad2_temp1 = zeros(Float64,size(LV,1),size(hind1,1))
        @tullio Qx[c] = QV[c,i] * x[i]
        @tullio grad2_temp1[c,k] = @inbounds(begin
        i,j = hind1[k]
                QV[c,i] * QV[c,j] * cos.(Qx[c]) end) grad=false fastmath=false
        @tullio grad2_temp[k] = grad2_temp1[c,k] * LV[c]
        @inbounds for i=1:size(hind1,1)
            j,k = hind1[i]
            grad2[j,k] = grad2_temp[i]
        end
        hessfull = Hermitian(grad2 + transpose(grad2) - Diagonal(grad2))
    end
    """
        hess!(x)
    Updatable version of hessian for optimiser
    """
    function hess!(hessian::Matrix, x::Vector)
        grad2 = zeros(ArbFloat,(h11,h11))
        hind1 = [[x,y]::Vector{Int64} for x=1:h11,y=1:h11 if x>=y]
        grad2_temp = zeros(ArbFloat,size(hind1,1))
        grad2_temp1 = zeros(Float64,size(LV,1),size(hind1,1))
        @tullio Qx[c] = QV[c,i] * x[i]
        @tullio grad2_temp1[c,k] = @inbounds(begin
        i,j = hind1[k]
                QV[c,i] * QV[c,j] * cos.(Qx[c]) end) grad=false
        @tullio grad2_temp[k] = grad2_temp1[c,k] * LV[c]
        @inbounds for i=1:size(hind1,1)
            j,k = hind1[i]
            grad2[j,k] = grad2_temp[i]
        hessian .= grad2 + transpose(grad2) - Diagonal(grad2)
        end
    end
    """
        grad(x)
    Gradient of potential at x for verifying G(x)>0 at minimum
    """
    grad(x) = vcat([dot(LV,QV[:,i] .* sin.(LinearAlgebra.mul!(Qx,QV,x))) for i ∈ 1:h11]...)
    #################
    ### Optimiser ###
    #################
    res = optimize(fitness,g!,hess!,
                x0, algo,
                Optim.Options(x_tol =minimum(abs.(LV)),g_tol =minimum(abs.(LV)),successive_f_tol = 5))
    Vmin = Optim.minimum(res)
    xmin = Optim.minimizer(res)
    #####################
    ## Verify minimum ###
    ### and output ######
    #####################
    if minimum(eigen(hess(xmin)).values) > 0 && maximum(abs.(grad(xmin))) < 1e-10
        vacua, θparalleltest, Qtilde = vacua_out(h11, QV, LV)
        a = (θparalleltest * xmin)/2π
        
        a_sign = Int.(sign.(a))
        a_log = [al ==0 ? 0 : Float64.(log10.(abs.(al))) for al in a]
        Vmin_sign = Int(sign(Vmin))
        Vmin_log = Float64(log10(abs(Vmin)))
        xmin_log = Float64.(log10.(abs.(xmin)))
        xmin_sign = Int.(sign.(xmin))
        a_sign, a_log, θparalleltest
    else
        (zeros(h11),zeros(h11),zeros(h11,h11))
    end
end



minimiser

In [6]:
"""
    main_timing(x0)
Function for looping over multiple initialisations (profiler)
"""
function main_timing(x0)
    h11,tri,cy = 10,1,1
    h11list = hcat([vcat(h11,tri,cy) for _=1:n]...)
    algo_hz = Newton(alphaguess = LineSearches.InitialHagerZhang(α0=1.0), linesearch = LineSearches.HagerZhang());
    return minimiser_timing(h11,tri,cy,LV,QV,x0,algo_hz)
end
"""
    main_test(x0)
Function for looping over multiple initialisations
"""
function main_test(x0)
    h11list = hcat([vcat(h11,tri,cy) for _=1:n]...)
    algo_hz = Newton(alphaguess = LineSearches.InitialHagerZhang(α0=1.0), linesearch = LineSearches.HagerZhang());
    return minimiser(h11,tri,cy,LV,QV,x0,algo_hz)
end

main_test

## Main -- vanilla

In [44]:
h11,tri,cy = 4,1,1
n=100 ##number of initialisations
x0 = hcat([rand(Uniform(0,2π),h11) .* rand(ArbFloat,h11) for _=1:n]...);
LV = pseudo_Lprime(h11,tri,cy)
QV = pseudo_Qprime(h11,tri,cy)
hcat(LV,QV)##print the L associated with the rows of Q

8×5 Matrix{ArbFloat}:
 1.0                                    1.0   0     0     0
 7.552215804662532327466667198699e-6    0     1.0   0     0
 6.015931487731031070320100003386e-11   0     0     1.0   0
 5.552701934419091651694588841875e-16   0     0     0     1.0
 9.049117489543092169361808203034e-21   4.0  -1.0   3.0   1.0
 3.331691844597027201072316476785e-26   3.0   2.0  -2.0  -3.0
 5.717491235910211302591814623364e-31  -1.0   0    -4.0  -5.0
 9.215129723695114457658300715506e-36  -2.0  -2.0  -1.0   5.0

In [45]:
@time res = mapslices(main_test,x0,dims=1);

  9.037418 seconds (27.79 M allocations: 750.353 MiB, 26.55% gc time, 26.47% compilation time)


In [46]:
vacua = vacua_out(h11,QV,LV)[1]
if vacua!=1
    println("There are ", vacua, " minima predicted by the analytic formula.")
else
    println("There is ", vacua, " minimum predicted by the analytic formula.")
end

There is 1.0 minimum predicted by the analytic formula.


In [47]:
### This tells us, within some tolerance, how many unique minima 
### have been found.  Some artefacts remain so check by eye in final set of minima
unique_mins = unique(mod.(round.(hcat([res[i][1] .* 10 .^res[i][2] for i=1:size(res,2)]...),digits=10),1),dims=2)

4×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0

In [48]:
## Check analytic computation of number of vacua
## is equal to number of unique minima found by optimization
vacua = vacua_out(h11,QV,LV)[1]
if vacua != size(unique_mins,2)
    println("The number of predicted vacua, ", vacua,", does not agree with the numerical computation, ", size(unique_mins,2))
else
    true
end

true

## Main -- shuffle

i.e. largest components of L not necessarily associated with the identity rows of Q

In [49]:
h11,tri,cy = 4,1,1
n=100 ##number of initialisations
x0 = hcat([rand(Uniform(0,2π),h11) .* rand(ArbFloat,h11) for _=1:n]...);
LV = pseudo_Lprime_shuffle(h11,tri,cy)
QV = pseudo_Qprime(h11,tri,cy)
hcat(LV,QV)##print the L associated with the rows of Q

8×5 Matrix{ArbFloat{128}}:
 1.0                                    1.0   0     0     0
 4.114006830718855309752977040851e-31   0     1.0   0     0
 3.023892675848558347396290620576e-36   0     0     1.0   0
 1.092520721149077741301773661594e-21   0     0     0     1.0
 1.547539025675650081438353599937e-26   1.0   2.0   0    -5.0
 4.305835529222196509204176692537e-6    2.0  -4.0  -1.0   4.0
 1.985048497827442768319773933899e-16  -5.0  -5.0  -1.0  -4.0
 3.147550396585364134216762323246e-11   0    -1.0   1.0  -5.0

In [50]:
@time res = mapslices(main_test,x0,dims=1);

  5.867010 seconds (23.09 M allocations: 557.510 MiB, 39.21% gc time)


In [51]:
vacua = vacua_out(h11,QV,LV)[1]
if vacua!=1
    println("There are ", vacua, " minima predicted by the analytic formula.")
else
    println("There is ", vacua, " minimum predicted by the analytic formula.")
end

There is 1.0 minimum predicted by the analytic formula.


In [52]:
### This tells us, within some tolerance, how many unique minima 
### have been found.  Some artefacts remain so check by eye in final set of minima
unique_mins = unique(mod.(round.(hcat([res[i][1] .* 10 .^res[i][2] for i=1:size(res,2)]...),digits=10),1),dims=2)

4×69 Matrix{Float64}:
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0
 0.897436  0.282051  0.358974  0.538462     0.589744  0.051282  0.820513
 0.461538  0.230769  0.384615  0.076923     0.846154  0.769231  0.307692
 0.512821  0.589744  0.205128  0.307692     0.051282  0.74359   0.897436

In [58]:
## Check analytic computation of number of vacua
## is equal to number of unique minima found by optimization
vacua = vacua_out(h11,QV,LV)[1]
if vacua != size(unique_mins,2)
    println("The number of predicted vacua, ", vacua,", does not agree with the numerical computation, ", size(unique_mins,2))
else
    true
end

The number of predicted vacua, 1.0, does not agree with the numerical computation, 69


## Main -- mixed

i.e. largest components of L not necessarily hierarchical

In [60]:
h11,tri,cy = 4,1,1
n=100 ##number of initialisations
x0 = hcat([rand(Uniform(0,2π),h11) .* rand(ArbFloat,h11) for _=1:n]...);
LV = pseudo_Lprime_mixed(h11,tri,cy)
QV = pseudo_Qprime(h11,tri,cy)
hcat(LV,QV)##print the L associated with the rows of Q

8×5 Matrix{ArbFloat}:
 1.0                                    1.0   0     0     0
 0.9000000000000000222044604925031      0     1.0   0     0
 0.8000000000000000444089209850063      0     0     1.0   0
 4.230033578650473390489461503907e-16   0     0     0     1.0
 4.321252257730562312917412952506e-21   2.0  -4.0  -1.0  -1.0
 8.258132448858494319892981467575e-26   0     5.0   5.0   3.0
 3.544258103003806319248836152178e-31   4.0   1.0  -4.0  -1.0
 1.715369536756665975882315775151e-37  -3.0  -2.0  -1.0   1.0

In [61]:
@time res = mapslices(main_test,x0,dims=1);

  7.079088 seconds (23.74 M allocations: 591.823 MiB, 28.48% gc time, 14.05% compilation time)


In [62]:
vacua = vacua_out(h11,QV,LV)[1]
if vacua!=1
    println("There are ", vacua, " minima predicted by the analytic formula.")
else
    println("There is ", vacua, " minimum predicted by the analytic formula.")
end

There is 1.0 minimum predicted by the analytic formula.


In [67]:
### This tells us, within some tolerance, how many unique minima 
### have been found.  Some artefacts remain so check by eye in final set of minima
unique_mins = unique(mod.(round.(hcat([res[i][1] .* 10 .^res[i][2] for i=1:size(res,2)]...),digits=10),1),dims=2);

In [66]:
## Check analytic computation of number of vacua
## is equal to number of unique minima found by optimization
vacua = vacua_out(h11,QV,LV)[1]
if vacua != size(unique_mins,2)
    println("The number of predicted vacua, ", vacua,", does not agree with the numerical computation, ", size(unique_mins,2))
else
    println("The number of predicted vacua is equal to the number computed using the numerical optimizer, i.e. ", vacua)
end

The number of predicted vacua is equal to the number computed using the numerical optimizer, i.e. 1.0
